In [1]:
import pygad
import numpy
import time
import pandas as pd
import sys
sys.path.append('../data')

In [2]:
data=pd.read_csv('../data/products.csv')

In [3]:
data.head()

,Product,Space,Price,Quantity
0,Refrigerator A,0.751000,999.90,3
1,Cell phone,0.000009,2199.12,2
2,TV 55,0.400000,4346.99,5
3,TV 50,0.290000,3999.90,5
4,TV 42,0.200000,2999.00,5


In [4]:
data2= data[['Product','Space', 'Price']]

In [5]:
data2

,Product,Space,Price
0,Refrigerator A,0.751000,999.90
1,Cell phone,0.000009,2199.12
2,TV 55,0.400000,4346.99
3,TV 50,0.290000,3999.90
4,TV 42,0.200000,2999.00
5,Notebook A,0.003500,2499.90
6,Ventilator,0.496000,199.90
7,Microwave A,0.042400,308.66
8,Microwave B,0.054400,429.90
9,Microwave C,0.031900,299.29


In [ ]:
data2.rename({'Product': 'thing'},{'Space':'weight'}, {'Price':'value'})

In [16]:
from collections import namedtuple


Thing= namedtuple('Thing',['name','weight','value'])
things= [
    Thing('Refrigerator A', 0.751, 999.9),
    Thing('Cell phone', 8.99e-06, 2199.12),
    Thing('TV 55', 0.4, 4346.99),
    Thing('TV 50', 0.29, 3999.9),
    Thing('TV 42', 0.2, 2999.0),
    Thing('Notebook A', 0.0035, 2499.9),
    Thing('Ventilator', 0.496, 199.9),
    Thing('Microwave A', 0.0424, 308.66),
    Thing('Microwave B', 0.0544, 429.9),
    Thing('Microwave C', 0.0319, 299.29),
    Thing('Refrigerator B', 0.635, 849.0),
    Thing('Refrigerator C', 0.87, 1199.89),
    Thing('Notebook B', 0.498, 1999.0),
    Thing('Notebook C', 0.527, 3999.0)
]

In [19]:
[Thing]

[__main__.Thing]

In [27]:

from collections import namedtuple
from functools import partial
from random import choices, randint, randrange, random
from typing import List, NamedTuple, Optional, Callable, Tuple

Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]
Thing= namedtuple('Thing',['name','weight','value'])

things= [
    Thing('Refrigerator A', 0.751, 999.9),
    Thing('Cell phone', 8.99e-06, 2199.12),
    Thing('TV 55', 0.4, 4346.99),
    Thing('TV 50', 0.29, 3999.9),
    Thing('TV 42', 0.2, 2999.0),
    Thing('Notebook A', 0.0035, 2499.9),
    Thing('Ventilator', 0.496, 199.9),
    Thing('Microwave A', 0.0424, 308.66),
    Thing('Microwave B', 0.0544, 429.9),
    Thing('Microwave C', 0.0319, 299.29),
    Thing('Refrigerator B', 0.635, 849.0),
    Thing('Refrigerator C', 0.87, 1199.89),
    Thing('Notebook B', 0.498, 1999.0),
    Thing('Notebook C', 0.527, 3999.0)
]

def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)
def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]
def fitness(genome:Genome, things:Thing, weight_limit:int) -> int:
    if len(genome) != len(things):
        raise ValueError("GENOME AND THINGS MUST BE OF SAME LENGTH")
    weight=0
    value=0
    for i, thing in enumerate(things):
        if genome [i]==1:
            weight +=thing.weight
            value += thing.value
            if weight > weight_limit:
                return 0
    return value
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]


def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum([fitness_func(genome) for genome in population])


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(genome) for genome in population],
        k=2
    )


def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation
    population=sorted(
        population,
        key=lambda genome:fitness_func(genome),
        reverse=True
    )

    return population, i 
population, generations=run_evolution(
    populate_func=partial(generate_population, size=10, genome_length=len(things)),
    fitness_func=partial(fitness, things=things, weight_limit=3),
    fitness_limit=14,
    generation_limit=100
 
)
def genome_to_things (genome: Genome, things: [Thing])->[Thing]:
    result =[]
    for i,thing in enumerate(things):
        if genome[i]==1:
            result+= [thing.name]
    return result

print (f"Number of generations: {generations}")
print(f"best solution:{genome_to_things(population[0], things)}")

Number of generations: 0
best solution:['Cell phone', 'TV 55', 'TV 50', 'Notebook A', 'Microwave A', 'Microwave C', 'Refrigerator C', 'Notebook B', 'Notebook C']
